In [1]:
!cp po8.db po9.db

In [50]:
'''
This file to edit the tables in the database
I have alreaady edited field_book where I dropped the duplicates and set the plant_name to be the primary key
'''

'\nThis file to edit the tables in the database\nI have alreaady edited field_book where I dropped the duplicates and set the plant_name to be the primary key\n'

In [2]:
'''
In the following I will edit rgb table
I will drop all the non relevant columns like:
    treatment
    plot 
    genotype 
I will keep the plant_name so it can be a foriegn key to field_book
That way we can lose redundant data and we can get this data from the 
field_book.

One conclusion I got from field book:
      {treatment, plot, genotype} hasn't changed overtime for each scan
'''
import sqlite3
import pandas as pd
from IPython.display import display

# global values
po8 = sqlite3.connect('po8.db')
po9 = sqlite3.connect('po9.db')
psii = pd.read_sql_query('SELECT * FROM psii', po8)


In [3]:
display(psii)

,index,Plot,F0,FM,FV,FV/FM,Date
0,0,MAC Field Scanner Season 10 Range 40 Column 30,13.960748,67.295251,53.334503,0.792545,2020-01-24
1,1,MAC Field Scanner Season 10 Range 41 Column 19,13.508523,59.143537,45.635015,0.771598,2020-01-24
2,2,MAC Field Scanner Season 10 Range 49 Column 21,12.215608,35.536869,23.321261,0.656255,2020-01-24
3,3,MAC Field Scanner Season 10 Range 38 Column 45,11.659572,41.517839,29.858267,0.719167,2020-01-24
4,4,MAC Field Scanner Season 10 Range 41 Column 7,13.102145,54.621915,41.519770,0.760130,2020-01-24
...,...,...,...,...,...,...,...
8635,8635,MAC Field Scanner Season 10 Range 51 Column 40,16.205196,82.241268,66.036072,0.802955,2020-02-20
8636,8636,MAC Field Scanner Season 10 Range 38 Column 10,15.725742,89.299231,73.573489,0.823898,2020-02-20
8637,8637,MAC Field Scanner Season 10 Range 39 Column 7,15.577724,79.912716,64.334992,0.805066,2020-02-20
8638,8638,MAC Field Scanner Season 10 Range 37 Column 40,16.357797,85.199033,68.841236,0.808005,2020-02-20


In [4]:
'''
ops:
1) drop columns
2) set primary keys
'''
#1) drop columns
drop_unwanted_columns = psii.drop(columns=['index'])
display(drop_unwanted_columns)


,Plot,F0,FM,FV,FV/FM,Date
0,MAC Field Scanner Season 10 Range 40 Column 30,13.960748,67.295251,53.334503,0.792545,2020-01-24
1,MAC Field Scanner Season 10 Range 41 Column 19,13.508523,59.143537,45.635015,0.771598,2020-01-24
2,MAC Field Scanner Season 10 Range 49 Column 21,12.215608,35.536869,23.321261,0.656255,2020-01-24
3,MAC Field Scanner Season 10 Range 38 Column 45,11.659572,41.517839,29.858267,0.719167,2020-01-24
4,MAC Field Scanner Season 10 Range 41 Column 7,13.102145,54.621915,41.519770,0.760130,2020-01-24
...,...,...,...,...,...,...
8635,MAC Field Scanner Season 10 Range 51 Column 40,16.205196,82.241268,66.036072,0.802955,2020-02-20
8636,MAC Field Scanner Season 10 Range 38 Column 10,15.725742,89.299231,73.573489,0.823898,2020-02-20
8637,MAC Field Scanner Season 10 Range 39 Column 7,15.577724,79.912716,64.334992,0.805066,2020-02-20
8638,MAC Field Scanner Season 10 Range 37 Column 40,16.357797,85.199033,68.841236,0.808005,2020-02-20


In [8]:
'''
From the data we can see tha each plant has one scan per date
let's check that out
'''
#check that date+name is not redundant
name_date = drop_unwanted_columns['Plot'] + drop_unwanted_columns['Date']
print(f'len(name_date): {len(name_date)}')
print(f'len(set(name_date)): {len(set(name_date))}')
if(len(name_date) == len(set(name_date))):
    print('equal')
#Thus, the combination of date name is unique

len(name_date): 8640
len(set(name_date)): 8640
equal


In [11]:
'''
commit new table drop_unwanted_columns
then edit with the script in the next cell
'''
schema = '''
CREATE TABLE IF NOT EXISTS "psii" (
"index" INTEGER,
  "Plot" TEXT NOT NULL,
  "F0" REAL,
  "FM" REAL,
  "FV" REAL,
  "FV/FM" REAL,
  "Date" TEXT NOT NULL,
  PRIMARY KEY("Plot", "Date")
);
'''
drop_unwanted_columns.to_sql('psii', po9, if_exists='replace')

8640

In [12]:
table = 'psii'
sql_command = f'''

PRAGMA foreign_keys=off;

BEGIN TRANSACTION;

ALTER TABLE {table} RENAME TO old_{table};

-- define the primary key constraint here
{schema}

INSERT INTO {table} SELECT * FROM old_{table};

DROP TABLE old_{table};

COMMIT;

PRAGMA foreign_keys=on;


'''
cursor = po9.cursor()
cursor.executescript(sql_command)